In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Github/MachineLearning/Module_3_Bank_Proj/Brandon

Mounted at /content/drive
/content/drive/MyDrive/Github/MachineLearning/Module_3_Bank_Proj/Brandon


In [ ]:
import pandas as pd

pd.read_csv("")